In [1]:
!pip install selenium

In [1]:
import requests
from bs4 import BeautifulSoup as bsp
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sqlite3

import json
import time
from datetime import datetime, timedelta
from selenium import webdriver

In [2]:
# PYTHON으로 SQLITE3의 DB에 데이터 담기
dbname='20210630_wadizDB_최종.db'
def insData(d, tblname='wadiztable_new'):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql='select nkey from '+tblname+' where nkey="'+d[-1]+'"'
        ckkey=cur.execute(sql).fetchall()
        if ckkey:
            print('-',end='')
        
        else:
            insql='insert into '+tblname+' (ncategory,ntitle,nnumofpic,ncommunity,nnews,ncontent,nsupport,namount,npercent,nkey) values (?,?,?,?,?,?,?,?,?,?)'
            cur.execute(insql,d)
            print('.',end='')

In [3]:
# DB에 와디즈의 게시물 제목과 내용 DB에 담기(연습으로 3개 게시물)
driver = webdriver.Chrome('C:\work2106\hackerton\chromedriver.exe')
# driver.get("https://www.wadiz.kr/web/wreward/category/313?keyword=&endYn=ALL&order=recommend") 
driver.get("https://www.wadiz.kr/web/wreward/category/297?keyword=&endYn=ALL&order=recommend") 

elements = driver.find_element_by_xpath(f'//*[@id="main-app"]/div[2]/div/div[5]/div[2]/div[2]/div/button')
time.sleep(0.5)
driver.execute_script("arguments[0].click();", elements)
time.sleep(0.5)

SCROLL_PAUSE_TIME = 1
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height

In [4]:
hrefs=[]
tables=driver.find_elements_by_class_name('CommonCard_image__vaqkf')
for table in tables:
    hrefs.append(table.get_attribute("href"))
len(hrefs)

794

In [5]:
hrefs

['https://www.wadiz.kr/web/campaign/detail/119816',
 'https://www.wadiz.kr/web/campaign/detail/116340',
 'https://www.wadiz.kr/web/campaign/detail/116678',
 'https://www.wadiz.kr/web/campaign/detail/100949',
 'https://www.wadiz.kr/web/campaign/detail/113876',
 'https://www.wadiz.kr/web/campaign/detail/107792',
 'https://www.wadiz.kr/web/campaign/detail/119588',
 'https://www.wadiz.kr/web/campaign/detail/117863',
 'https://www.wadiz.kr/web/campaign/detail/115708',
 'https://www.wadiz.kr/web/campaign/detail/109754',
 'https://www.wadiz.kr/web/campaign/detail/113650',
 'https://www.wadiz.kr/web/campaign/detail/112284',
 'https://www.wadiz.kr/web/campaign/detail/119604',
 'https://www.wadiz.kr/web/campaign/detail/115698',
 'https://www.wadiz.kr/web/campaign/detail/113817',
 'https://www.wadiz.kr/web/campaign/detail/117830',
 'https://www.wadiz.kr/web/campaign/detail/118634',
 'https://www.wadiz.kr/web/campaign/detail/116761',
 'https://www.wadiz.kr/web/campaign/detail/117180',
 'https://ww

In [7]:
for i in range(len(hrefs)):
    url = hrefs[i]
    reqheader = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0;Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'} 
    src=requests.get(url,headers=reqheader)
    cont=src.content.strip() 
    html=bsp(cont,'html.parser')

    # category
    category=html.select('p.title-info em')
    ncategory=category[0].text
    
    # title
    title=html.select('h2.title a')
    ntitle=title[0].text
    
    #img
    img_url = []
    imgs = html.find_all('img')
    for img in imgs:
        iurl = img.get('data-src')

        if iurl != None:
            img_url.append(iurl)
        else:
            iurl = img.get('src')
            img_url.append(iurl)
    nnumofpics = len(img_url)

    # contents and summary
    contents=html.select('.inner-contents p')
    summary=html.select('.campaign-summary')
    cont=[]
    
    for content in contents:
        cont.append(content.text)
    ncontent=summary[0].text+' '.join(cont)
    
    # community(댓글수)
    comm = html.find_all('span','count-total comment-total')
    if len(comm) == 0:
        ncommunity = 0
    else:
        ncommunity = comm[0].text
    
    # news(새소식)
    news = html.find_all('span','count-total')
    nnews = news[0].text

    # num of supporters
    support=html.select('p.total-supporter strong')
    nsupport=support[0].text
    
    # amount
    amount=html.select('p.total-amount strong')
    namount=amount[0].text
    
    # percentage
    percent=html.select('p.achievement-rate strong')
    npercent=percent[0].text

    # key
    nkey= url
    
    

    d=(ncategory,ntitle,nnumofpics,ncommunity,nnews,ncontent,nsupport,namount,npercent,nkey)
    insData(d)

................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................--........................................................................................................................

In [ ]:
len(comm)